In [1]:
# pytorch is consuming so much ram, so, I've selected the tensorflow

import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as ts
import tensorflow.keras as ks
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical

pd.set_option('display.max_columns', 784)

#### Normalizing and reshaping the dataset

In [2]:
df_path         = '../input/digit-recognizer/'
im_shape = (28, 28, 1)

out_df = pd.read_csv(df_path + 'sample_submission.csv')
out_df.set_index('ImageId', inplace = True)

def prepare_data(df: pd.DataFrame, train = False):
    x_train = y_train = None
    
    if train:
        y_train = df['label']
        y_train = to_categorical(y_train, num_classes = 10)
        x_train = df.drop('label', axis=1, inplace=False)
    else:
        x_train = df
    
    # There is no semantic loss but convolutions **cores** will contain sane numbers
    x_train = x_train / 255
    
    # (rows, 28, 28, 1) 
    x_train = x_train.values.reshape(-1, *im_shape)
    
    return x_train, y_train

x_test, _        = prepare_data(pd.read_csv(df_path + 'test.csv'))
x_train, y_train = prepare_data(pd.read_csv(df_path + 'train.csv'),  train=True)

x_train_splitted, x_test_splitted, y_train_splitted, y_test_splitted = train_test_split(
    x_train, y_train, test_size = 0.2
)

#### Creating CNN model using the keras api

In [3]:
from keras.preprocessing.image import ImageDataGenerator

# augmentation setup
mnist_datagen = ImageDataGenerator(
        samplewise_center              = False, 
        featurewise_std_normalization  = False,  
        samplewise_std_normalization   = False,  
        zca_whitening                  = True,  
        horizontal_flip                = False,  
        vertical_flip                   = False,
    
        rotation_range                 = 10,  
        width_shift_range              = 0.1,  
        height_shift_range             = 0.1, 
)

/opt/conda/lib/python3.7/site-packages/keras_preprocessing/image/image_data_generator.py:337: UserWarning: This ImageDataGenerator specifies `zca_whitening`, which overrides setting of `featurewise_center`.
  warnings.warn('This ImageDataGenerator specifies '


In [4]:
# learning options
epoch_count = 150
prefered_batch_size  = 256

model_options = {
    'optimizer': 'adam',
    'metrics':   'accuracy',                  
    'loss':      'categorical_crossentropy'  # awesome loss-function for multiclass classification
}
lrelu = lambda x: ks.layers.LeakyReLU(alpha = 0.01)(x)

conv_setup = {
    'kernel_size': 3,
    'strides':     1,               
    'activation': 'relu'
}

lr_model = ks.Sequential()
# -----------------------------------------------------------------  # C1
lr_model.add(ks.layers.Conv2D(
    filters     = 64,              
    input_shape = im_shape,
    **conv_setup
))
lr_model.add(ks.layers.BatchNormalization())

lr_model.add(ks.layers.Conv2D(
    filters     = 64,
    **conv_setup
))
lr_model.add(ks.layers.BatchNormalization())

lr_model.add(ks.layers.MaxPool2D()) 
lr_model.add(ks.layers.Dropout(0.4))

# -----------------------------------------------------------------  # C2
lr_model.add(ks.layers.Conv2D(
    filters     = 128,
    **conv_setup
))
lr_model.add(ks.layers.BatchNormalization())

lr_model.add(ks.layers.Conv2D(
    filters     = 128,
    **conv_setup
))
lr_model.add(ks.layers.BatchNormalization())

lr_model.add(ks.layers.MaxPool2D())
lr_model.add(ks.layers.Dropout(0.4))

# -----------------------------------------------------------------  # Classifier part
lr_model.add(ks.layers.Flatten())   
lr_model.add(ks.layers.Dense(
    1024, activation = 'swish'
))
lr_model.add(ks.layers.Dropout(0.4))
lr_model.add(ks.layers.BatchNormalization())

# -----------------------------------------------------------------  # Output layer
lr_model.add(ks.layers.Dense(
    10, activation='softmax'        # 10 classes
))

# Assembly model
lr_model.compile(**model_options)
lr_model.summary()

2022-06-16 07:58:14.688944: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-16 07:58:14.789507: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-16 07:58:14.790477: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-16 07:58:14.791711: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 64)        640       
_________________________________________________________________
batch_normalization (BatchNo (None, 26, 26, 64)        256       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 64)        36928     
_________________________________________________________________
batch_normalization_1 (Batch (None, 24, 24, 64)        256       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 12, 12, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 10, 10, 128)       7

In [5]:
# train our model
lr_model.fit_generator(
    mnist_datagen.flow(x_train_splitted, y_train_splitted, batch_size = prefered_batch_size),
    verbose = 1,
    epochs = 100,
    steps_per_epoch= x_train_splitted.shape[0] // prefered_batch_size,
    validation_data = (x_test_splitted, y_test_splitted),    
)

/opt/conda/lib/python3.7/site-packages/keras/engine/training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '
/opt/conda/lib/python3.7/site-packages/keras_preprocessing/image/image_data_generator.py:720: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
/opt/conda/lib/python3.7/site-packages/keras_preprocessing/image/image_data_generator.py:739: UserWarning: This ImageDataGenerator specifies `zca_whitening`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn('This ImageDataGenerator specifies '
2022-06-16 07:58:17.593455: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Pas

Epoch 1/100


2022-06-16 07:58:19.350036: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


131/131 [==============================] - 16s 67ms/step - loss: 0.3865 - accuracy: 0.8788 - val_loss: 14.6610 - val_accuracy: 0.1067
Epoch 2/100
131/131 [==============================] - 8s 62ms/step - loss: 0.1113 - accuracy: 0.9649 - val_loss: 12.3652 - val_accuracy: 0.1067
Epoch 3/100
131/131 [==============================] - 9s 69ms/step - loss: 0.0767 - accuracy: 0.9763 - val_loss: 6.5125 - val_accuracy: 0.1207
Epoch 4/100
131/131 [==============================] - 8s 64ms/step - loss: 0.0637 - accuracy: 0.9794 - val_loss: 0.3973 - val_accuracy: 0.8821
Epoch 5/100
131/131 [==============================] - 9s 70ms/step - loss: 0.0552 - accuracy: 0.9818 - val_loss: 0.0438 - val_accuracy: 0.9871
Epoch 6/100
131/131 [==============================] - 8s 64ms/step - loss: 0.0495 - accuracy: 0.9851 - val_loss: 0.0266 - val_accuracy: 0.9921
Epoch 7/100
131/131 [==============================] - 8s 64ms/step - loss: 0.0440 - accuracy: 0.9856 - val_loss: 0.0326 - val_accuracy: 0.9907
E

In [6]:
# predict classes
y_preds = lr_model.predict(x_test)
y_preds_classified = np.argmax(y_preds, axis=1)
y_preds_classified[:10]

array([2, 0, 9, 0, 3, 7, 0, 3, 0, 3])

In [7]:
# upload results 
out_df['Label'] = y_preds_classified
out_df.to_csv('outer.csv')